# 👕 Test d’inférence serverless : segmentation de vêtements sur images réelles

Objectif de ce notebook :
- **Prouver** que l’appel API au modèle SegFormer-clothes de Hugging Face fonctionne bien sur tes données locales (images d’influenceurs à segmenter).
- Réaliser un test complet de la chaîne technique : lecture de l’image, transmission à l’API, décodage du résultat et premières visualisations de masques.
- Afficher un signal visuel “tout est au vert” si l’inférence fonctionne, pour rassurer l’équipe et amorcer la suite : calcul de métriques et visualisation avancée.

Ce diagnostic est clé pour valider que la configuration (token, dépendances, sécurité) est solide et que l’API répond bien sur tes images brutes.

In [1]:
# Ce bloc s’assure que le .env est bien chargé et que le token HF est accessible partout
import os
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path(".env")
load_dotenv(dotenv_path=dotenv_path)
HF_TOKEN = os.environ.get("HF_TOKEN")
assert HF_TOKEN, "❌ Le token HF n'a pas été trouvé dans l'environnement !"
model_id = "sayeed99/segformer_b3_clothes"

AssertionError: ❌ Le token HF n'a pas été trouvé dans l'environnement !

In [2]:
# On charge une image du dossier data et on l’affiche (utile pour la traçabilité)
from PIL import Image

test_image_path = "data/raw/images/image_0.png"
img = Image.open(test_image_path)
img.show()
print(f"✅ Image test chargée : {test_image_path} (taille : {img.size})")

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/images/image_0.png'

In [3]:
# Envoi en POST (binaire), header adapté selon le type d’image
import requests

with open(test_image_path, "rb") as f:
    image_bytes = f.read()

headers = {
    "Authorization": f"Bearer {HF_TOKEN}",
    "Content-Type": "image/png",
}

resp = requests.post(
    f"https://api-inference.huggingface.co/models/{model_id}",
    headers=headers,
    data=image_bytes,
    timeout=20,
)

print(f"Statut retour : {resp.status_code}")
print("Extrait JSON brut :", resp.content[:120], "…")

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/images/image_0.png'

In [4]:
# Lecture et affichage du contenu de la réponse (labels, présence de masques)
import json

try:
    output = resp.json()
    labels_predits = [elt["label"] for elt in output if "label" in elt]
    print("Labels prédits :", labels_predits)
    masks = [elt for elt in output if "mask" in elt]
    print(f"Nb de masques (base64) reçus : {len(masks)}")
except Exception as e:
    print(f"❌ Erreur lors du décodage de la réponse : {e}")

❌ Erreur lors du décodage de la réponse : name 'resp' is not defined


In [5]:
# Affichage stylé d’un message de validation ou d’erreur
from IPython.display import display, Markdown

if resp.status_code in (200, 503) and isinstance(output, list) and "mask" in output[0]:
    display(Markdown("<span style='color:green;font-weight:bold;font-size:1.2em;'>✅ Tout est au vert&nbsp;: API, token, requête et format résultat OK !</span>"))
else:
    display(Markdown(f"<span style='color:red;font-weight:bold;'>❌ Erreur de validation API&nbsp;: code {resp.status_code}<br/>{resp.content[:200]}</span>"))

NameError: name 'resp' is not defined

In [6]:
# Bonus : visualisation rapide d’un masque de segmentation
import base64
from PIL import Image
from io import BytesIO

mask_b64 = output[0]["mask"]
mask_bytes = base64.b64decode(mask_b64)
mask_img = Image.open(BytesIO(mask_bytes))
mask_img.show()
print("Premier masque décodé et affiché (base64 → PNG).")

NameError: name 'output' is not defined